# EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import io
import os
from scipy import signal

In [4]:
#load dataset
#3 patients, pre and post stroke
path = r"C:\Users\jamesjun\Desktop\Simone\BCI_Comp"

P1_pre_training = io.loadmat(path + '\P1_pre_training.mat')
P1_post_training = io.loadmat(path + '\P1_post_training.mat')
P1_pre_test = io.loadmat(path + '\P1_pre_test.mat')
P1_post_test = io.loadmat(path + '\P1_post_test.mat')

P2_pre_training = io.loadmat(path + '\P2_pre_training.mat')
P2_post_training = io.loadmat(path + '\P2_post_training.mat')
P2_pre_test = io.loadmat(path + '\P2_pre_test.mat')
P2_post_test = io.loadmat(path + '\P2_post_test.mat')

P3_pre_training = io.loadmat(path + '\P3_pre_training.mat')
P3_post_training = io.loadmat(path + '\P3_post_training.mat')
P3_pre_test = io.loadmat(path + '\P3_pre_test.mat')
P3_post_test = io.loadmat(path + '\P3_post_test.mat')

In [5]:
np.shape(P1_pre_training['trig'])

(271816, 1)

In [6]:
(unique, counts) = np.unique(P1_pre_training['trig'], return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[    -1,  79889],
       [     0, 110007],
       [     1,  81920]], dtype=int64)

In [7]:
for m in [P1_pre_training, P1_post_training, P1_pre_test, P1_post_test, 
          P2_pre_training, P2_post_training, P2_pre_test, P2_post_test, 
          P3_pre_training, P3_post_training, P3_pre_test, P3_post_test]:
    print(np.shape(m['y']))    #16 channels

(271816, 16)
(197343, 16)
(204560, 16)
(194088, 16)
(223112, 16)
(216720, 16)
(199552, 16)
(233576, 16)
(205536, 16)
(206504, 16)
(248488, 16)
(218432, 16)
